In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
class AE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(AE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc3 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc3(h) 
    
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        z = self.encoder(x.view(-1, 784))
        x_hat = self.decoder(z)
        return x_hat

# build model
ae = AE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=8)
if torch.cuda.is_available():
    ae.cuda()

In [3]:
ae

AE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [4]:
optimizer = optim.Adam(ae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(x_hat, x):
    BCE = F.binary_cross_entropy(x_hat, x.view(-1, 784), reduction='sum')
    return BCE 

In [5]:
def train(epoch):
    ae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        x_hat = ae(data)
        loss = loss_function(x_hat, data)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [6]:
def test():
    ae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            x_hat = ae(data)
            
            # sum up batch loss
            test_loss += loss_function(x_hat, data).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
for epoch in range(1, 21):
    train(epoch)
    test()

C:\Users\DuyPhuong\anaconda3\envs\duyphuong\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 543.513359
Train Epoch: 1 [10000/60000 (17%)]	Loss: 176.053242
Train Epoch: 1 [20000/60000 (33%)]	Loss: 132.345957
Train Epoch: 1 [30000/60000 (50%)]	Loss: 124.868613
Train Epoch: 1 [40000/60000 (67%)]	Loss: 112.772891
Train Epoch: 1 [50000/60000 (83%)]	Loss: 106.606699
====> Epoch: 1 Average loss: 139.8297
====> Test set loss: 102.0910
Train Epoch: 2 [0/60000 (0%)]	Loss: 97.008203
Train Epoch: 2 [10000/60000 (17%)]	Loss: 109.224063
Train Epoch: 2 [20000/60000 (33%)]	Loss: 98.476709
Train Epoch: 2 [30000/60000 (50%)]	Loss: 95.093750
Train Epoch: 2 [40000/60000 (67%)]	Loss: 96.470010
Train Epoch: 2 [50000/60000 (83%)]	Loss: 94.467305
====> Epoch: 2 Average loss: 98.5334
====> Test set loss: 94.5135
Train Epoch: 3 [0/60000 (0%)]	Loss: 93.203965
Train Epoch: 3 [10000/60000 (17%)]	Loss: 100.745156
Train Epoch: 3 [20000/60000 (33%)]	Loss: 93.240059
Train Epoch: 3 [30000/60000 (50%)]	Loss: 89.219072
Train Epoch: 3 [40000/60000 (67%)]	Loss: 85.725400
Train 

In [21]:
with torch.no_grad():
    z = torch.randn(1, 8).cuda()
    
    print(z)
    z = torch.Tensor([[ 0.0,  1.0,  1.0,  0.0,  0.0,  0.0, -0.0, -0.0]]).cuda()
    sample = ae.decoder(z).cuda()
    
    save_image(sample.view(1, 1, 28, 28), './samples/sample2_' + '.png')

tensor([[-0.3433,  1.2624, -0.7182,  0.0895, -1.5500,  0.6599,  0.1492,  1.0275]],
       device='cuda:0')


In [13]:
import numpy as np
import pandas as pd
z_list = []
with torch.no_grad():
    for data, _ in test_loader:
        data = data.cuda()
        z = ae.encoder(data.view(-1, 784))
        z_list.append(z.cpu().detach().numpy())
z = np.concatenate(z_list, axis=0)
pd.DataFrame(z).to_csv("./latent_space_z.csv", header=None, index=False)


In [ ]:
[ 0.3298,  1.3922,  1.6839,  0.9390,  0.5467,  0.3180, -0.1014, -0.1166] 6